In [2]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import optuna
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')

In [8]:
pd.set_option('display.max_columns', None)

In [10]:
## modelling for silicon :
si_data = pd.read_csv('../data/cleaned/silicon.csv')
si_bandgap = pd.read_csv('../data/cleaned/bandgapSi.csv')

In [11]:
si_data.isnull().sum()

atomic_numbers_mean            0
atomic_numbers_max             0
atomic_numbers_min             0
atomic_numbers_std             0
volume_per_atom                0
                              ..
average_cationic_radius_std    0
average_anionic_radius_mean    0
average_anionic_radius_max     0
average_anionic_radius_min     0
average_anionic_radius_std     0
Length: 95, dtype: int64

In [12]:
si_data.shape

(5000, 95)

In [13]:
si_data.head(2)

,atomic_numbers_mean,atomic_numbers_max,atomic_numbers_min,atomic_numbers_std,volume_per_atom,empty_volume,metals_fraction,distance_matrix_mean,distance_matrix_max,distance_matrix_min,distance_matrix_std,van_der_waals_radius_mean,van_der_waals_radius_max,van_der_waals_radius_min,van_der_waals_radius_std,electrical_resistivity_mean,electrical_resistivity_max,electrical_resistivity_min,electrical_resistivity_std,velocity_of_sound_mean,velocity_of_sound_max,velocity_of_sound_min,velocity_of_sound_std,reflectivity_mean,reflectivity_max,reflectivity_min,reflectivity_std,poissons_ratio_mean,poissons_ratio_max,poissons_ratio_min,poissons_ratio_std,molar_volume_mean,molar_volume_max,molar_volume_min,molar_volume_std,thermal_conductivity_mean,thermal_conductivity_max,thermal_conductivity_min,thermal_conductivity_std,melting_point_mean,melting_point_max,melting_point_min,melting_point_std,critical_temperature_mean,critical_temperature_max,critical_temperature_min,critical_temperature_std,superconduction_temperature_mean,superconduction_temperature_max,superconduction_temperature_min,superconduction_temperature_std,liquid_range_mean,liquid_range_max,liquid_range_min,liquid_range_std,bulk_modulus_mean,bulk_modulus_max,bulk_modulus_min,bulk_modulus_std,youngs_modulus_mean,youngs_modulus_max,youngs_modulus_min,youngs_modulus_std,brinell_hardness_mean,brinell_hardness_max,brinell_hardness_min,brinell_hardness_std,rigidity_modulus_mean,rigidity_modulus_max,rigidity_modulus_min,rigidity_modulus_std,vickers_hardness_mean,vickers_hardness_max,vickers_hardness_min,vickers_hardness_std,density_of_solid_mean,density_of_solid_max,density_of_solid_min,density_of_solid_std,coefficient_of_linear_thermal_expansion_mean,coefficient_of_linear_thermal_expansion_max,coefficient_of_linear_thermal_expansion_min,coefficient_of_linear_thermal_expansion_std,average_ionic_radius_mean,average_ionic_radius_max,average_ionic_radius_min,average_ionic_radius_std,average_cationic_radius_mean,average_cationic_radius_max,average_cationic_radius_min,average_cationic_radius_std,average_anionic_radius_mean,average_anionic_radius_max,average_anionic_radius_min,average_anionic_radius_std
0,13.000000,14,12,1.000000,18.856730,223.616458,0.083333,3.124381,4.791231,0.0,1.164230,1.73,1.73,1.73,0.0,4.400000e-08,4.400000e-08,4.400000e-08,0.0,4602.0,4602.0,4602.0,0.0,74.0,74.0,74.0,0.0,0.29,0.29,0.29,0.0,14.00,14.00,14.00,0.0,160.0,160.0,160.0,0.0,923.0,923.0,923.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,440.0,440.0,440.0,0.0,45.0,45.0,45.0,0.0,45.0,45.0,45.0,0.0,260.0,260.0,260.0,0.0,17.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,1738.0,1738.0,1738.0,0.0,0.000008,0.000008,0.000008,0.0,0.86,0.86,0.86,0.0,0.86,0.86,0.86,0.0,0.0,0.0,0.0,0.0
1,12.588235,41,8,9.052519,14.965699,1524.191713,0.009804,6.787983,12.008606,0.0,2.803602,2.18,2.18,2.18,0.0,1.520000e-07,1.520000e-07,1.520000e-07,0.0,3480.0,3480.0,3480.0,0.0,0.0,0.0,0.0,0.0,0.40,0.40,0.40,0.0,10.83,10.83,10.83,0.0,54.0,54.0,54.0,0.0,2750.0,2750.0,2750.0,0.0,0.0,0.0,0.0,0.0,9.25,9.25,9.25,0.0,2267.0,2267.0,2267.0,0.0,170.0,170.0,170.0,0.0,105.0,105.0,105.0,0.0,736.0,736.0,736.0,0.0,38.0,38.0,38.0,0.0,1320.0,1320.0,1320.0,0.0,8570.0,8570.0,8570.0,0.0,0.000007,0.000007,0.000007,0.0,0.82,0.82,0.82,0.0,0.82,0.82,0.82,0.0,0.0,0.0,0.0,0.0
